<a href="https://colab.research.google.com/github/Yaswanthyarra/MicroexpressionClassificationUsingCNNandGeneticAlgorithm/blob/main/RandomSearchBasedTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/CASMEII'

/content/drive/My Drive/CASMEII


In [3]:
dataset_dir = '/content/drive/My Drive/CASMEII/Cropped-updated/Cropped'

In [4]:
import numpy as np
import pandas as pd
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

# define constants
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 32
NUM_EPOCHS = 60

# load the onset and apex frame indices from the Excel sheet
coding_file = pd.read_excel('/content/drive/My Drive/CASMEII/CASME2-coding-20140508.xlsx', sheet_name='Sheet1')

class_labels = coding_file['Estimated Emotion'].values

# define a function to extract the onset and apex frames for each expression
def extract_frames(sequence_path,sequence, subject_num):
    expression_images = []

    #To obtain the onset and apex frame numbers for the given subject and the corresponding sequence from the xlsheet
    subject_data = coding_file.loc[coding_file['Subject'] == subject_num]
    sequence_data = subject_data.loc[subject_data['Filename'] == sequence]
    onset_frame = sequence_data['OnsetFrame'].iloc[0]
    apex_frame = sequence_data['ApexFrame'].iloc[0]
                                                                                                                                                                 
    #To obtain the path to frames corresponding to the apex and onset frame numbers from the sequence of frames for the subject
    onset_frame_path = os.path.join(sequence_path,'reg_img' + str(onset_frame) + '.jpg')
    apex_frame_path = os.path.join(sequence_path, 'reg_img' + str(apex_frame) + '.jpg')
    print(subject_num,apex_frame_path)
    print(subject_num,onset_frame_path)
   
    #To obtain the apex and apex frames from path
    onset_frame = cv2.imread(onset_frame_path)
    apex_frame = cv2.imread(apex_frame_path)
    onset_frame = cv2.cvtColor(onset_frame, cv2.COLOR_BGR2GRAY)
    apex_frame = cv2.cvtColor(apex_frame, cv2.COLOR_BGR2GRAY)
    
    #To apply histogram equalisation to the frames to to enhance the contrast of an image by redistributing the pixel intensities. 
    onset_frame = cv2.equalizeHist(onset_frame)
    apex_frame = cv2.equalizeHist(apex_frame)

    #Resizing extracted frames
    onset_frame = cv2.resize(onset_frame, IMAGE_SIZE)
    apex_frame = cv2.resize(apex_frame, IMAGE_SIZE)

    #Stacking the apex and onset frames together and and appending to the expressin images list
    expression_images.append(np.stack([onset_frame, apex_frame], axis=-1))
    
    return np.array(expression_images)

# create a data generator for the image data
data_generator = ImageDataGenerator(rescale=1./255)

#Setting up several image augmentation parameters that can be used to generate augmented images 
exp_frame_augmentation = {
    'rotation_range': 10,
    'width_shift_range': 0.1,
    'height_shift_range': 0.1,
    'shear_range': 0.1,
    'zoom_range': 0.1,
    'horizontal_flip': True,
    'brightness_range': [0.5, 1.5],
}


# load the expression folders and extract the onset and apex frames for each subject

#converting the categorical class labels of a classification problem into numerical form
label_dict = {'happiness': 0, 'disgust': 1, 'repression': 2, 'surprise': 3, 'fear': 4, 'others': 6,'sadness':5}

expression_images = []
expression_labels = []

#Looping through all the subjects in the dataset
for subject in os.listdir(dataset_dir):

    subject_num = int(subject[3:])
    subject_dir = os.path.join(dataset_dir, subject)

    #Looping through each sequence for the subject
    for sequence in os.listdir(subject_dir):
        
        sequence_path = os.path.join(subject_dir, sequence)

        #Extracting the onset and apex frames by invoking the extract_frames method
        expression_frames = extract_frames(sequence_path, sequence, subject_num)

        #Obtaining the expression label for the sequence from the xlsheet and appending it to expression_label 
        subject_data = coding_file.loc[coding_file['Subject'] == subject_num]
        sequence_data = subject_data.loc[subject_data['Filename'] == sequence]
        exp_label_category = sequence_data['Estimated Emotion']
        expression_label = label_dict[exp_label_category.values[0]]

        #Adding the extracted frames and the correspoing class label to expression_images and expression_labels
        expression_images.append(expression_frames)
        expression_labels.append(expression_label)

        # Generating augmented images for the fear class
        if expression_label == 4:  # if fear class
            
            for i in range(30):
              print(i,subject_num)

              #Generating the augmented apex and onset frames
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)
              
              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)
        
        # Generating augmented images for the sadness class
        if expression_label == 5:  # if sadness class
           
            for i in range(9):
              print(i,subject_num)
              
              #Generating the augmented apex and onset frames
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)
              

              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)  

        # generate augmented images for the happiness class
        if expression_label == 0:  # if happiness class
        
            for i in range(1):
              print(i,subject_num)

              #Generating the augmented apex and onset frames
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)
              
              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)  

        # Generate augmented images for the repression class
        if expression_label == 2:  # if repression class
          
            for i in range(1):
              print(i,subject_num)
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)

              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)  
        
        # Generate augmented images for the surprise class
        if expression_label == 3:  # if surprise class
            
            
            for i in range(1):
              print(i,subject_num)

              #Generating the augmented apex and onset frames
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)

              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)  


        

        
# convert the data to numpy arrays 
expression_images = np.concatenate(expression_images, axis=0)
expression_labels = np.array(expression_labels)

#To obtain the number of class labels
num_classes = len(np.unique(expression_labels))

#Stratified Spliting of data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(expression_images, expression_labels, test_size=0.2,stratify=expression_labels,  random_state=42)



25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP09_02/reg_img101.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP09_02/reg_img76.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP10_01/reg_img129.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP10_01/reg_img101.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP03_01/reg_img81.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP03_01/reg_img56.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP03_02/reg_img101.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP03_02/reg_img51.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP10_10/reg_img91.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP10_10/reg_img61.jpg
0 25
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP18_04f/reg_img116.jpg
25 /content/drive/My Drive/CASME

In [5]:
from keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# define the function to create the model
def create_model(learning_rate=0.01, dropout_rate=0.0):
  
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 2)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        
        Dense(num_classes, activation='softmax')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# create the KerasClassifier object
model = KerasClassifier(build_fn=create_model, verbose=1)

# define the hyperparameters to tune
param_dist = {
    'learning_rate': [0.0001, 0.001, 0.0008,0.002],
    'dropout_rate': [0.3, 0.4, 0.2],
    'epochs': [60, 50, 40],
    'batch_size': [128, 32, 64]
}

# perform random search cross-validation
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3)
random_search.fit(train_images, train_labels)

# evaluate the best model on the test set
best_model = random_search.best_estimator_
best_keras_model = best_model.model
test_loss, test_accuracy = best_keras_model.evaluate(test_images, test_labels)
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")

<ipython-input-5-aada3a655ecb>:30: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=1)


Epoch 1/50
4/4 [==============================] - 4s 824ms/step - loss: 78.2810 - accuracy: 0.1341
Epoch 2/50
4/4 [==============================] - 6s 1s/step - loss: 18.8364 - accuracy: 0.1504
Epoch 3/50
4/4 [==============================] - 3s 638ms/step - loss: 2.8169 - accuracy: 0.2154
Epoch 4/50
4/4 [==============================] - 2s 425ms/step - loss: 2.0863 - accuracy: 0.2439
Epoch 5/50
4/4 [==============================] - 2s 423ms/step - loss: 1.8691 - accuracy: 0.2480
Epoch 6/50
4/4 [==============================] - 2s 427ms/step - loss: 1.8150 - accuracy: 0.2846
Epoch 7/50
4/4 [==============================] - 2s 426ms/step - loss: 1.7192 - accuracy: 0.3293
Epoch 8/50
4/4 [==============================] - 2s 609ms/step - loss: 1.5890 - accuracy: 0.4106
Epoch 9/50
4/4 [==============================] - 3s 734ms/step - loss: 1.4859 - accuracy: 0.4431
Epoch 10/50
4/4 [==============================] - 2s 422ms/step - loss: 1.3634 - accuracy: 0.4959
Epoch 11/50
4/4 [===

1/1 [==============================] - 0s 434ms/step - loss: 0.9119 - accuracy: 0.6911
Epoch 1/40
2/2 [==============================] - 2s 837ms/step - loss: 33.3039 - accuracy: 0.1220
Epoch 2/40
2/2 [==============================] - 2s 781ms/step - loss: 33.7093 - accuracy: 0.2276
Epoch 3/40
2/2 [==============================] - 2s 787ms/step - loss: 17.1322 - accuracy: 0.1301
Epoch 4/40
2/2 [==============================] - 2s 815ms/step - loss: 5.2420 - accuracy: 0.1829
Epoch 5/40
2/2 [==============================] - 2s 772ms/step - loss: 3.2917 - accuracy: 0.1951
Epoch 6/40
2/2 [==============================] - 3s 2s/step - loss: 2.2374 - accuracy: 0.1992
Epoch 7/40
2/2 [==============================] - 3s 1s/step - loss: 1.9360 - accuracy: 0.2154
Epoch 8/40
2/2 [==============================] - 2s 824ms/step - loss: 1.8542 - accuracy: 0.2846
Epoch 9/40
2/2 [==============================] - 2s 799ms/step - loss: 1.7414 - accuracy: 0.3293
Epoch 10/40
2/2 [=================

1/1 [==============================] - 0s 401ms/step - loss: 0.5183 - accuracy: 0.7967
Epoch 1/40
8/8 [==============================] - 2s 203ms/step - loss: 75.9638 - accuracy: 0.1301
Epoch 2/40
8/8 [==============================] - 3s 376ms/step - loss: 2.1838 - accuracy: 0.1870
Epoch 3/40
8/8 [==============================] - 3s 321ms/step - loss: 1.8489 - accuracy: 0.2764
Epoch 4/40
8/8 [==============================] - 2s 247ms/step - loss: 1.7488 - accuracy: 0.2927
Epoch 5/40
8/8 [==============================] - 2s 240ms/step - loss: 1.6613 - accuracy: 0.3211
Epoch 6/40
8/8 [==============================] - 2s 241ms/step - loss: 1.5187 - accuracy: 0.4309
Epoch 7/40
8/8 [==============================] - 2s 258ms/step - loss: 1.4096 - accuracy: 0.4756
Epoch 8/40
8/8 [==============================] - 3s 329ms/step - loss: 1.3026 - accuracy: 0.5163
Epoch 9/40
8/8 [==============================] - 3s 382ms/step - loss: 1.1333 - accuracy: 0.6138
Epoch 10/40
8/8 [=============

In [6]:
print(random_search.best_params_)

{'learning_rate': 0.001, 'epochs': 60, 'dropout_rate': 0.3, 'batch_size': 64}
